<a href="https://colab.research.google.com/github/alexkociubinski/Monte-Carlo-Lattice-/blob/main/monteCarloLattice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import math
import random

#Define T variable and size. Create and initialize lattice
T = 2.3 #Temperature
size = 8
P = 1/8 #Hole Ratio
num_holes = int(size * size * P)
U = 2 #interaction strength between neighboring spings
t = 1.0 # hopping energy for paired hole movements

lattice = [[random.choice([1, -1]) for _ in range(size)] for _ in range(size)]

def can_place_hole(lattice, r, c):
  if lattice[r][c] == 0:
    return False

  neighbors = [
        ((r - 1) % size, c),
        ((r + 1) % size, c),
        (r, (c - 1) % size),
        (r, (c + 1) % size),
    ]

  for nr, nc in neighbors:
    if lattice[nr][nc] == 0:
      return False
  return True

holes_placed = 0
while holes_placed < num_holes:
    row = random.randint(0, size-1)
    col = random.randint(0, size-1)
    if lattice[row][col] == 0:
      continue

    if can_place_hole(lattice, row, col):
        lattice[row][col] = 0
        holes_placed += 1



#create calculate_energy function. Get values of all the neighbors of selected element in lattice. Calculate the energy.
def calculate_energy(lattice, r, c):
    size = len(lattice)
    current_index = lattice[r][c]

    if current_index == 0:
        return 0

    neighbors = [
        lattice[(r - 1) % size][c],
        lattice[(r + 1) % size][c],
        lattice[r][(c - 1) % size],
        lattice[r][(c + 1) % size],
    ]

    neighbors_sum = sum(n for n in neighbors if n != 0)

    energy = U * current_index * neighbors_sum
    return energy


def find_hole_pairs(lattice):
    pairs = []
    for r in range(size):
        for c in range(size):
            if lattice[r][c] != 0: #check to start at a hole
              continue

            c2 = (c + 3) % size
            if lattice[r][c2] == 0:
                pairs.append(((r, c), (r, c2), 'H'))

            r2 = (r + 3) % size
            if lattice[r2][c] == 0:
                pairs.append(((r, c), (r2, c), 'V'))

    unique = []
    seen = set()
    for(p1, p2, orient) in pairs:
        key = (frozenset([p1, p2]), orient)
        if key not in seen:
            seen.add(key)
            unique.append((p1, p2, orient))
    return unique


def is_valid_hole_position(lattice, r, c, moving_holes):
    above = (r - 1) % size
    below = (r + 1) % size


    if(above, c) not in moving_holes and lattice[above][c] == 0:
        return False

    if(below, c) not in moving_holes and lattice[below][c] == 0:
        return False

    return True


def pair_region_energy(lattice, h1, h2):
    r1, c1 = h1
    r2, c2 = h2

    affected = set()
    for (r,c) in [h1, h2]:
        for(nr, nc) in [
            ((r - 1) % size, c),
            ((r + 1) % size, c),
            (r, (c - 1) % size),
            (r, (c + 1) % size),
        ]:
            if(nr, nc) != h1 and (nr, nc) != h2:
                affected.add((nr, nc))
    total = sum(calculate_energy(lattice, r, c) for (r, c) in affected)
    return total


def attempt_pair_move(lattice):
    pairs = find_hole_pairs(lattice)

    if not pairs:
        return

    (r1, c1), (r2, c2), orientation = random.choice(pairs)

    if orientation == 'H':
        dr, dc = random.choice([(-1, 0), (1,0)]) #up or down
    else:
        dr, dc = random.choice([(0, -1), (0, 1)]) #left or right

    nr1, nc1 = (r1 + dr) % size, (c1 + dc) % size
    nr2, nc2 = (r2 + dr) % size, (c2 + dc) % size

    moving = {(r1, c1), (r2, c2)}
    if lattice[nr1][nc1] == 0 and (nr1, nc1) not in moving:
        return
    if lattice[nr2][nc2] == 0 and (nr2, nc2) not in moving:
        return

    if not is_valid_hole_position(lattice, nr1, nc1, moving):
        return
    if not is_valid_hole_position(lattice, nr2, nc2, moving):
        return

    E_before = pair_region_energy(lattice, (r1, c1), (r2, c2))

    spin_at_nr1 = lattice[nr1][nc1]
    spin_at_nr2 = lattice[nr2][nc2]

    lattice[r1][c1] = lattice[nr1][nc1]  # spin at destination comes back
    lattice[r2][c2] = lattice[nr2][nc2]
    lattice[nr1][nc1] = 0  # hole moves forward
    lattice[nr2][nc2] = 0

    E_after = pair_region_energy(lattice, (nr1, nc1), (nr2, nc2))

    E_after -= t
    delta_E = E_after - E_before

    if delta_E < 0:
        pass
    else:
        probability = math.exp(-delta_E / T)
        if random.random() >= probability:
            lattice[nr1][nc1] = lattice[r1][c1]
            lattice[nr2][nc2] = lattice[r2][c2]
            lattice[r1][c1] = 0
            lattice[r2][c2] = 0



steps = 3005000
for step in range(steps):

    move_type = random.random()

    if move_type < 0.75:
        # ---- SPIN FLIP (original code, unchanged) ----
        row = random.randint(0, size - 1)
        col = random.randint(0, size - 1)

        if lattice[row][col] == 0:
            continue

        previous_energy = calculate_energy(lattice, row, col)
        lattice[row][col] *= -1
        new_energy = calculate_energy(lattice, row, col)
        change_energy = new_energy - previous_energy

        if change_energy < 0:
            pass
        else:
            accept_probability = math.exp(-change_energy / T)
            if random.random() >= accept_probability:
                lattice[row][col] *= -1

    else:
        # ---- PAIRED HOLE MOVE (new) ----
        attempt_pair_move(lattice)



#Print out lattice
for row in lattice:
    print(" ".join(f"{x:+2d}" for x in row))

+0 +1 +0 +1 -1 +1 -1 +1
+1 -1 +1 -1 +1 -1 +0 -1
-1 +1 -1 +0 -1 +1 -1 +1
+1 +0 +1 -1 +1 -1 +1 -1
-1 +1 -1 +1 -1 +1 -1 +1
+1 -1 +1 +0 +1 -1 +0 -1
-1 +0 -1 +1 -1 +1 -1 +1
+1 -1 +1 -1 +1 -1 +1 -1


In [11]:
test_lattice = [
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 0, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1],  # horizontal pair at (3,1) and (3,4)
    [1, 1, 0, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1, 1, 1, 1],
]

print("BEFORE:")
for row in test_lattice:
    print(" ".join(f"{x:+2d}" for x in row))

# Force many pair move attempts
for _ in range(10000):
    attempt_pair_move(test_lattice)

print("\nAFTER:")
for row in test_lattice:
    print(" ".join(f"{x:+2d}" for x in row))

# What to check:
# - The two 0s should have moved to a different row (up or down)
# - They should still be in the SAME columns (1 and 4) — horizontal pairs only move vertically
# - They should still be exactly 3 apart
# - No two 0s should be vertically adjacent

BEFORE:
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +0 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +0 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +1 +1

AFTER:
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +0 +1
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +0 +1
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +1 +1
+1 +1 +1 +1 +1 +1 +1 +1


In [9]:
def check_no_stacking(lattice):
    violations = []
    for r in range(size):
        for c in range(size):
            if lattice[r][c] == 0:
                below = (r + 1) % size
                if lattice[below][c] == 0:
                    violations.append(((r,c), (below, c)))
    return violations

# Add this check inside your MC loop to catch the exact step it breaks
steps = 3005000
for step in range(steps):
    move_type = random.random()
    if move_type < 0.75:
        # ... spin flip code ...
        pass
    else:
        attempt_pair_move(lattice)

    # Check every 10000 steps
    if step % 10000 == 0:
        v = check_no_stacking(lattice)
        if v:
            print(f"STACKING VIOLATION at step {step}: {v}")
            break
else:
    print("No stacking violations found in entire run!")

KeyboardInterrupt: 